In [23]:
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

import json

import mygene
mg = mygene.MyGeneInfo()
 
metaDir = os.path.join(os.path.dirname('__file__'), '..', 'metadata')
p3Dir = os.path.join(os.path.dirname('__file__'), '..', '3')

df = pd.read_excel(os.path.join(metaDir, 'Gene_Map.xlsx'), sheetname='Map') 
    
# Getting the Gene Map
gene_list = list(df['Gene'])
uniprot_list = list(df['UniprotAC'])
gene_id_list = list(df['GeneID'])

with open(os.path.join(metaDir, 'mapping.json')) as mapping:
    map_dict = json.load(mapping)

data = map_dict["rows"]

def get_uniprot(gene_id):
    
    uniprot_id = ''
    
    for i in range(len(data)):
        row = data[i]
        if(row['cell'][1] == gene_id):
            uniprot_id = row['cell'][4]
    
    return uniprot_id

In [24]:
# 4.1 - Seed genes interactome


# Read Biogrid data
# Gene A, Gene B
bio_df = pd.read_csv(os.path.join(p3Dir, 'BioGrid.csv'))

seed_gene_intercom = []


for index,row in bio_df.iterrows():
    
    gene1 = row['GeneA']
    gene2 = row['GeneB']
    
    if (gene1 in gene_id_list and gene2 in gene_id_list):
        gene1_index = gene_id_list.index(gene1)
        gene2_index = gene_id_list.index(gene2)
        
        intercom_row = [gene_list[gene1_index], gene_list[gene2_index], uniprot_list[gene1_index], uniprot_list[gene2_index], "BioGrid"]
        seed_gene_intercom.append(intercom_row)


# Read IID data
# Symbol 1, Symbol 2
iid_df = pd.read_csv(os.path.join(p3Dir, 'IID.csv'))

for index,row in iid_df.iterrows():
    
    symbol1 = row['symbol1']
    symbol2 = row['symbol2']
    
    if (symbol1 in gene_list and symbol2 in gene_list):
        gene1_index = gene_list.index(symbol1)
        gene2_index = gene_list.index(symbol2)
        
        intercom_row = [symbol1, symbol2, uniprot_list[gene1_index], uniprot_list[gene2_index], "IID"]
        seed_gene_intercom.append(intercom_row)

        
columns_list = ["GeneA","GeneB", "UniprotA", "UniprotB", "DataSource"]

intercom_df = pd.DataFrame(seed_gene_intercom, columns=columns_list)

intercom_df.to_csv('4.1.csv',encoding='utf8', index=False)



In [25]:
#4.2 - Union interactome

bio_df = pd.read_csv(os.path.join(p3Dir, 'BioGrid.csv'))

seed_gene_intercom = []


for index,row in bio_df.iterrows():
    
    gene1 = row['GeneA']
    gene2 = row['GeneB']
    
    if (gene1 in gene_id_list or gene2 in gene_id_list):
        symbol1 = ''
        symbol2 = ''
        uniprot1 = ''
        uniprot2 = ''
        
        if gene1 not in gene_id_list:
            symbol1 = mg.getgene(gene1)['symbol']
            uniprot1 = get_uniprot(symbol1)
            #print("Uniprot 1 Bio: ")
            #print(symbol1)
        else:
            gene1_index = gene_id_list.index(gene1) 
            symbol1 = gene_list[gene1_index]
            uniprot1 = uniprot_list[gene1_index]
        
        if gene2 not in gene_id_list:
            symbol2 = mg.getgene(gene2)['symbol']
            uniprot2 = get_uniprot(symbol2)
            #print("Uniprot 2 Bio: ")
            #print(uniprot2)
        else:
            gene2_index = gene_id_list.index(gene2)
            symbol2 = gene_list[gene2_index]
            uniprot2 = uniprot_list[gene2_index]
        
        intercom_row = [symbol1, symbol2, uniprot1, uniprot2, "BioGrid"]
        seed_gene_intercom.append(intercom_row)
        
# Read IID data
# Symbol 1, Symbol 2
iid_df = pd.read_csv(os.path.join(p3Dir, 'IID.csv'))

for index,row in iid_df.iterrows():
    
    gene1 = row['symbol1']
    gene2 = row['symbol2']
    
    if (gene1 in gene_list or gene2 in gene_list):
        symbol1 = ''
        symbol2 = ''
        uniprot1 = ''
        uniprot2 = ''
        
        if gene1 not in gene_list:
            symbol1 = gene1
            uniprot1 = get_uniprot(symbol1)
        else:
            gene1_index = gene_list.index(gene1) 
            symbol1 = gene_list[gene1_index]
            uniprot1 = uniprot_list[gene1_index]
        
        if gene2 not in gene_list:
            symbol2 = gene2
            uniprot2 = get_uniprot(symbol2)
        else:
            gene2_index = gene_list.index(gene2)
            symbol2 = gene_list[gene2_index]
            uniprot2 = uniprot_list[gene2_index]
        
        intercom_row = [symbol1, symbol2, uniprot1, uniprot2, "IID"]
        seed_gene_intercom.append(intercom_row)

columns_list = ["GeneA","GeneB", "UniprotA", "UniprotB", "DataSource"]

intercom_df = pd.DataFrame(seed_gene_intercom, columns=columns_list)

intercom_df.to_csv('4.2.csv',encoding='utf8', index=False)

In [26]:
#4.3 - Intersection interactome

bio_df = pd.read_csv(os.path.join(p3Dir, 'BioGrid.csv'))

seed_gene_intercom_bio = []
seed_gene_intercom_iid = []


for index,row in bio_df.iterrows():
    
    gene1 = row['GeneA']
    gene2 = row['GeneB']
    
    if (gene1 in gene_id_list or gene2 in gene_id_list):
        symbol1 = ''
        symbol2 = ''
        uniprot1 = ''
        uniprot2 = ''
        
        if gene1 not in gene_id_list:
            symbol1 = gene1
            uniprot1 = get_uniprot(symbol1)
        else:
            gene1_index = gene_id_list.index(gene1) 
            symbol1 = gene_list[gene1_index]
            uniprot1 = uniprot_list[gene1_index]
        
        if gene2 not in gene_id_list:
            symbol2 = gene2
            uniprot2 = get_uniprot(symbol2)
        else:
            gene2_index = gene_id_list.index(gene2)
            symbol2 = gene_list[gene2_index]
            uniprot2 = uniprot_list[gene2_index]
        
        intercom_row = [symbol1, symbol2, uniprot1, uniprot2, "BioGrid"]
        seed_gene_intercom_bio.append(intercom_row)


# Read IID data
# Symbol 1, Symbol 2
iid_df = pd.read_csv(os.path.join(p3Dir, 'IID.csv'))

for index,row in iid_df.iterrows():
    
    gene1 = row['symbol1']
    gene2 = row['symbol2']
    
    if (gene1 in gene_list or gene2 in gene_list):
        
        symbol1 = ''
        symbol2 = ''
        uniprot1 = ''
        uniprot2 = ''
        
        if gene1 not in gene_list:
            symbol1 = gene1
            uniprot1 = get_uniprot(symbol1)
        else:
            gene1_index = gene_list.index(gene1) 
            symbol1 = gene_list[gene1_index]
            uniprot1 = uniprot_list[gene1_index]
        
        if gene2 not in gene_list:
            symbol2 = gene2
            uniprot2 = get_uniprot(symbol2)
        else:
            gene2_index = gene_list.index(gene2)
            symbol2 = gene_list[gene2_index]
            uniprot2 = uniprot_list[gene2_index]
        
        intercom_row = [symbol1, symbol2, uniprot1, uniprot2, "IID"]
        seed_gene_intercom_iid.append(intercom_row)
   
        
columns_list = ["GeneA","GeneB", "UniprotA", "UniprotB", "DataSource"]

bio_df = pd.DataFrame(seed_gene_intercom_bio, columns=columns_list)
iid_df = pd.DataFrame(seed_gene_intercom_iid, columns=columns_list)

bioList = []
bioList = bio_df["GeneA"]
bioList = set(bioList.append(bio_df["GeneB"]))

iiDList = []
iiDList = iid_df["GeneA"]
iiDList = set(iiDList.append(iid_df["GeneB"]))

intersect_list = list(bioList.intersection(iiDList))

if 'Missing' in intersect_list: intersect_list.remove('Missing')

union_df = pd.read_csv('4.2.csv')

i = 1
for seed in intersect_list:
    if(i==1):
        newDF = union_df[(union_df['GeneA']==seed) | (union_df['GeneB']==seed)]
    else:
        newDF = newDF.append(union_df[(union_df['GeneA']==seed) | (union_df['GeneB']==seed)])
    i=2

del newDF['DataSource']
    
newDF.to_csv('4.3.csv',encoding='utf8', index=False)
